Section 1 : Import Libraries 


In [41]:
import requests #This helps us to fetch data from API's
import pandas as pd # for handling and analysing data
import numpy as np #for numerical operation
from sklearn.model_selection import train_test_split#to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder # to convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # models for classification and regression task
from sklearn.metrics import mean_squared_error # to measure the accuracy of our prediction 
from datetime import datetime,timedelta # to handle date and time
import pytz

In [42]:
API_KEY= 'f56a46e05986fbfd15778c52e89945d4' 
BASE_URL='https://api.openweathermap.org/data/2.5/' # base url for making API request


1. Fetch Current WEather Data

In [43]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"#easily insert variable into a string format
    response=requests.get(url)# send the get request to API
    data = response.json()
    return {
        'city':data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_max']),
        'humidity' : round(data['main']['humidity']),
        'description' : data['weather'][0]['description'],
        'country' : data['sys']['country'],
        'wind_gust_dir':data['wind']['deg'],
        'pressure':data['main']['pressure'],
        'WindGustSpeed':data['wind']['speed']
}

2.Read Historical data

In [44]:
def read_historical_data(filename):
    df=pd.read_csv(filename) # load csv file into dataframe
    df =df.dropna() # remove rows with missing values
    df = df.drop_duplicates()
    return df

3.Prepare data For training

In [45]:
def prepare_data(data):
    le = LabelEncoder() # create a labelencoder instance
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])
    # define the feature and target variables
    X =data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']] # Features variables
    y= data['RainTomorrow'] # target variable
    return X,y, le  # return feature variable ,target variable and label encoder


4.Train Rain Prediction Model

In [46]:
def train_rain_model(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    model=RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Mean Squared Error For  Rain Model")
    print(mean_squared_error(y_test,y_pred))
    return model 

5.Prepare regression data

In [47]:
def prepare_regression_data(data,features):
    X,y=[],[]#Initialize list for feature and target values


    for i in range(len(data)-1):
        X.append(data[features].iloc[i])
        y.append(data[features].iloc[i+1])
    X=np.array(X).reshape(-1,1)
    y=np.array(y)
    return X,y
    

6.Train Regression Model

In [48]:
def train_regression_model(X,y):
    model= RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(X,y)
    return model

Predict Future

In [49]:
def predict_future(model,current_value):
    predictions=[current_value]
    for i in range(5):
        next_value=model.predict(np.array([[predictions[-1]]]))

        predictions.append(next_value[0])
    return predictions[1:]





Weather Analysis Function                          ts

In [56]:
def weather_view():
    city=input("enter the city name:")
    current_weather=get_current_weather(city)
    #load historical data
    historical_data=read_historical_data('C:/Users/Sarnalika/OneDrive/Documents/Project/WEATHER_FORECAST/weather.csv')
    #prepare and train the rain prediction model
    X,y,le=prepare_data(historical_data)
    rain_model=train_rain_model(X,y)

    #map wind direction to campass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N",0,11.25),("NNE",11.25,33.75),("NE",33.75,56.25),
        ("ENE",56.25,78.75),("E",78.75,101.25),("ESE",101.25,123.75),
        ("SE",123.75,146.25),("SSE",146.25,168.75),("S",168.75,191.25),
        ("SSW",191.25,213.75),("SW",213.75,236.25),("WSW",236.25,258.75),
        ("W",258.75,281.25),("WNW",281.25,303.75),("NW",303.75,326.25),
        ("NNW",326.25,348.75)
    ]


    compass_direction= next(point for point ,start,end in compass_points if start<=wind_deg<end)
    compass_direction_encoded=le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data={
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir':compass_direction_encoded,
        'WindGustSpeed':current_weather['WindGustSpeed'],
        'Humidity':current_weather['humidity'],
        'Pressure':current_weather['pressure'],
        'Temp':current_weather['current_temp'],
        
    }
    current_df=pd.DataFrame([current_data])

    #rain prediction
    rain_prediction=rain_model.predict(current_df)[0]
    #prepare regression model for temperature and humidity
    X_temp,y_temp = prepare_regression_data(historical_data,'Temp')
    X_hum,y_hum = prepare_regression_data(historical_data,'Humidity')
    temp_model= train_regression_model(X_temp,y_temp)
    hum_model= train_regression_model(X_hum,y_hum)
    # predict future temperature and humidity
    future_temp=predict_future(temp_model,current_weather['temp_min'])
    future_humidity=predict_future(hum_model,current_weather['humidity'])
    #prepare time for future predictions
    time_zone=pytz.timezone('Asia/Kolkata')
    now=datetime.now(time_zone)
    next_hour=now + timedelta(hours=1)
    next_hour=next_hour.replace(minute=0,second=0,microsecond=0)

    future_times=[(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]
    #display results

    print(f"city:{city},{current_weather['country']}")
    print(f"Current Temperature:{current_weather['current_temp']}")
    print(f"Feels_Like:{current_weather['feels_like']}")
    print(f"Minimum Temperature:{current_weather['temp_min']}°C")
    print(f"Maximum Temperature:{current_weather['temp_max']}°C")
    print(f"Humidity:{current_weather['humidity']}")
    print(f"Weather Prediction:{current_weather['description']}")
    print(f"Rain Prediction:{'Yes' if rain_prediction else 'No'}")

    print("\n Future Temperature Predictions:")

    for time,temp in zip(future_times,future_temp):
        print(f"{time}:{round(temp,1)}°C")

    print("\nFuture Humidity Predictions:")

    for time,humidity in zip(future_times,future_humidity):
        print(f"{time}:{round(humidity,1)}%")

weather_view()   
    

Mean Squared Error For  Rain Model
0.1506849315068493
city:Kolkata,IN
Current Temperature:26
Feels_Like:26
Minimum Temperature:26°C
Maximum Temperature:26°C
Humidity:61
Weather Prediction:haze
Rain Prediction:Yes

 Future Temperature Predictions:
14:00:28.2°C
15:00:20.4°C
16:00:22.2°C
17:00:22.7°C
18:00:25.0°C

Future Humidity Predictions:
14:00:58.3%
15:00:54.2%
16:00:58.1%
17:00:54.2%
18:00:58.1%
